In [ ]:

# https://stackoverflow.com/questions/59936599/how-do-i-use-decision-tree-regressor-on-new-data-python-pandas-sklearn
# 决策树—回归 
# 决策树是一种基本的分类与回归方法，本文叙述的是回归部分。回归决策树主要指CART(classification and regression tree)算法，内部结点特征的取值为“是”和“否”， 为二叉树结构。

import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor


# Path of the file to read
iowa_file_path = 'train.csv'

home_data = pd.read_csv(iowa_file_path)
#Simplify data to remove useless info
SimpleTable=home_data[['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd','SalePrice']]
# Create target object and call it y # input target value
y = home_data.SalePrice 
# Create X input columns names to be analysed
features = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
X = home_data[features]

# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=0, test_size=0.8, train_size=0.2)


# Specify Model
iowa_model = DecisionTreeRegressor(random_state=0)
# Fit Model
iowa_model.fit(train_X, train_y)

# Make validation predictions and calculate mean absolute error
val_predictions = iowa_model.predict(val_X)

val_mae = mean_absolute_error(val_predictions, val_y)
print("Validation MAE: {:,.0f}".format(val_mae))


def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

# to find best number of leaves
candidate_max_leaf_nodes = [10, 20, 50, 100, 200, 400] # start with big numbers are work your way down
for max_leaf_nodes in candidate_max_leaf_nodes:
    my_mae=get_mae(max_leaf_nodes,train_X,val_X,train_y,val_y)
    print("MAX leaf nodes: %d \t\t Mean Absolute Error:%d" %(max_leaf_nodes,my_mae))




scores = {leaf_size: get_mae(leaf_size, train_X, val_X, train_y, val_y) for leaf_size in candidate_max_leaf_nodes}

best_tree_size = min(scores, key=scores.get)
print(best_tree_size)


#run on all data and put back into data fram 
final_model=DecisionTreeRegressor(max_leaf_nodes=best_tree_size,random_state=0)
final_model.fit(X,y)
final_model.predict(X)

final_predictions = final_model.predict(X)
finaltableinput = {'Predicted_Price':final_predictions}
finaltable = pd.DataFrame(finaltableinput)
SimpleTable.head()

jointable = SimpleTable.join(finaltable)

#export data with predicted values to csv
jointable.to_csv('newdata4.csv')
